## Step 5 - Transfer learning

In [11]:
# Running Tensorboard

%load_ext tensorboard
%tensorboard --logdir=./mylogs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 23112), started 1 day, 0:11:32 ago. (Use '!kill 23112' to kill it.)

In [18]:
# Creating image generators

from keras.preprocessing.image import ImageDataGenerator

image_shape = (224, 224, 3) # I want the images to be square and not to big to speed up training.
test_split_size = 0.15
batch_size = 128
rescale_factor = 1/255

# Define the parameters for data augmentation
train_datagen_2 = ImageDataGenerator(
    rescale=rescale_factor,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


# Create the training dataset
train_dataset_2 = train_datagen_2.flow_from_directory(
    'data/flowers_cleaned_split/train',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

val_datagen_2 = ImageDataGenerator(rescale=1./255)

# Create the val dataset
val_dataset_2 = val_datagen_2.flow_from_directory(
    'data/flowers_cleaned_split/val/',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen_2 = ImageDataGenerator(rescale=1./255)

# Create the test dataset
test_dataset_2 = test_datagen_2.flow_from_directory(
    'data/flowers_cleaned_split/test/',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


print(f'Number Of Images In Training Dataset : {str(train_dataset_2.samples)}')
print(f'Number Of Images In Validation Dataset : {str(val_dataset_2.samples)}')
print(f'Number Of Images In Validation Dataset : {str(test_dataset_2.samples)}')
print(f'Total number of images: {train_dataset_2.samples+val_dataset_2.samples+test_dataset_2.samples}')

Found 2915 images belonging to 5 classes.
Found 624 images belonging to 5 classes.
Found 628 images belonging to 5 classes.
Number Of Images In Training Dataset : 2915
Number Of Images In Validation Dataset : 624
Number Of Images In Validation Dataset : 628
Total number of images: 4167


In [26]:
import tensorflow as tf

def generator(directory_iterator):
    for x, y in directory_iterator:
        yield x, y

train_dataset_tf = tf.data.Dataset.from_generator(
    lambda: generator(train_dataset_2),
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 224, 224, 3], [None, 3])
)

val_dataset_tf = tf.data.Dataset.from_generator(
    lambda: generator(val_dataset_2),
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 224, 224, 3], [None, 3])
)

In [27]:
import keras_tuner as kt
from keras.applications import MobileNetV3Large, MobileNetV3Small
from keras import Input
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Normalization
from keras.optimizers import Adam


def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=1, max_value=3, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=512)
    learning_rate = hp.Float(
        "learning_rate", min_value=1e-4, max_value=1e-2, default=1e-3, sampling="log"
    )

    base_model = MobileNetV3Large(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)

    for _ in range(n_hidden):
        x = Dense(n_neurons, activation="relu")(x)

    outputs = Dense(5, activation="softmax")(x)

    model = Model(inputs, outputs)

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)

    def fit(self, hp, model, dataset, **kwargs):
        # Check if normalization is required and the dataset is a TensorFlow dataset
        if hp.Boolean("normalize") and hasattr(dataset, "map"):
            norm_layer = Normalization()
            dataset = dataset.map(lambda x, y: (norm_layer(x), y))
        
        # Fit the model
        return model.fit(dataset, **kwargs)


hyperband_tuner = kt.Hyperband(
    MyClassificationHyperModel(),
    objective="val_accuracy",
    seed=42,
    max_epochs=10,
    factor=3,
    hyperband_iterations=2,
    overwrite=True,
    directory="hyperparams",
    project_name="hyperband",
)

In [28]:
from pathlib import Path
from keras.callbacks import TensorBoard, EarlyStopping
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = TensorBoard(root_logdir)
early_stopping_cb = EarlyStopping(patience=2)
hyperband_tuner.search(train_dataset_2, epochs=10, validation_data = val_dataset_2,callbacks=[early_stopping_cb, tensorboard_cb] )


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |n_hidden
35                |35                |n_neurons
0.00065625        |0.00065625        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


In [14]:
# Mobilenet, liten modell
from keras.applications import MobileNetV3Large, MobileNetV3Small
from keras import Input
from keras.models import Model
from keras.layers import GlobalAveragePooling2D

def create_mobilenet_feature_extraction_model():
    base_model = MobileNetV3Large(weights='imagenet', include_top=False)
    # Freeze the base model
    base_model.trainable = False
    
    # Create a new model on top
    inputs = Input(shape=(224, 244, 3))
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = (Dropout(0.2))(x)
    outputs = Dense(5, activation='softmax')(x)
    model = Model(inputs, outputs)
    
    # Compile and train the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

print("hej")


hej


In [15]:
model2 = create_mobilenet_feature_extraction_model()

NameError: name 'Dropout' is not defined

In [ ]:
model2.summary()

In [ ]:
# Callbacks

# Defining tensorboard callback for logging and visualization
def get_run_logdir(custom_text="", root_logdir="my_logs"):
    formatted_time = strftime("run_%Y_%m_%d_%H_%M_%S")
    if custom_text:
        custom_text = "_" + custom_text
    return Path(root_logdir) / (formatted_time + custom_text)


run_logdir = get_run_logdir("transfer_learning_2_mobilenet_Large_averagepooling")

tensorboard_cb = TensorBoard(run_logdir, profile_batch=(100, 200))

# Defining earlystopping callback to stop training if model isnt getting better
early_stop_cb = EarlyStopping(monitor="val_loss", patience=10)
run_logdir

In [ ]:
# Training model
history = model2.fit(train_dataset_2,epochs=10,steps_per_epoch = steps_per_epoch,
                    validation_data = val_dataset_2, validation_steps=validation_steps,
                    verbose=True, callbacks=[early_stop_cb, tensorboard_cb])